# Guannan Shen BIOS 7718 Bio Image  HW1 

In [3]:
# loading required packages
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
# install open cv 
# anaconda show menpo/opencv3
import cv2 as cv
from scipy.odr import odrpack
from sklearn import linear_model
%matplotlib inline

## References
https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.random.multivariate_normal.html  

https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.identity.html  

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

https://lectures.quantecon.org/py/ols.html

http://blog.rtwilson.com/orthogonal-distance-regression-in-python/

https://docs.scipy.org/doc/scipy/reference/odr.html

http://mathworld.wolfram.com/LeastSquaresFittingPerpendicularOffsets.html

https://stackoverflow.com/questions/29731726/how-to-calculate-a-gaussian-kernel-matrix-efficiently-in-numpy

## Question 1  
We want to fit a line by minimizing the errors: the sum of squared vertical distances between data points and the line; or we will minimize the sum of squared perpendicular distances between data points and the line

### 1 Data Generation
Using equation, $1+6x^o −y^o = 0$ to generate n = 30 data points, where $x^o_i = 0, 1, ..., 29$. The noise is assumed to be independent and identically distributed (i.i.d.) and it follows a multivariate normal distribution with mean $µ = 0$ and covariance $Σ = 5^2I$ (I is the 2 × 2 identify matrix).


Get the clean data points. 

In [68]:
# two column DF 
n = 30
v_o = pd.DataFrame({'Inter' : [0]*30,
                    'x' : range(n),
                   'y' : [6*x + 1 for x in range(n)]} )
v_o.head(3)


,Inter,x,y
0,0,0,1
1,0,1,7
2,0,2,13


Get the noise data, with np.random.multivariate_normal. Concatenate a list of 100 DataFrame. 

In [69]:
mean1 = [0,0]
cov1 = 25*np.identity(2)
# np.random.seed(7)
# set random seed
v = []
for i in range(100):
    np.random.seed(i)
    noise = pd.DataFrame(np.random.multivariate_normal(mean1, cov1, 30).round(0),
                        columns=['x', 'y'])
    noise['Inter'] = [1]*30
    v_i = noise + v_o
    v.append(v_i)
v = pd.concat(v, axis=0)
v.head(10)

,Inter,x,y
0,1,9.0,3.0
1,1,6.0,18.0
2,1,11.0,8.0
3,1,8.0,18.0
4,1,3.0,27.0
5,1,6.0,38.0
6,1,10.0,38.0
7,1,9.0,45.0
8,1,15.0,48.0
9,1,11.0,51.0


### Fitting ordinal least squares and orthogonal distance regression
For the ordinal least squares, given $y=X\beta+\epsilon$, this is to find the minima of $(y-X\beta)^2$. 

Apply matrix calculus, this is to let the derivative of (with respect to the $\beta$)
$$(\beta^TX^TX\beta - 2y^TX\beta + yTy)$$
which is $$(2X^TX\beta - 2X^Ty = 0)$$
If $(X^TX)$ is invertible. We have the solution of $\beta$
$$\beta = (X^TX)^{-1}X^Ty$$

### Fitting with an outlier data